<a href="https://colab.research.google.com/github/cyrildiagne/instagram-3d-photo/blob/master/colab_worker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Restart the Runtime after running this cell.

In [0]:
!pip install google-cloud-pubsub google-cloud-storage

Upload the insta3d-colab-key.json that was generated by the script `script/create_colab_key.sh` before running this cell.

In [1]:
# TODO: Upload the colab serviceaccount key file.
%env GOOGLE_APPLICATION_CREDENTIALS=/content/insta3d-colab-key.json

PROJECT_ID = '' # TODO: Set your GCP Project ID.
BUCKET_NAME = '' # TODO: Set your GCP Bucket Name.

env: GOOGLE_APPLICATION_CREDENTIALS=/content/insta3d-colab-key.json


# Initialize 3d-photo-inpainting

In [0]:
# Install dependencies
!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install opencv-python==4.2.0.32
!pip3 install vispy==0.6.4
!pip3 install moviepy==1.0.2
!pip3 install transforms3d==0.3.1
!pip3 install networkx==2.3
!sudo apt install sed

# Get code from custom fork & retrieve checkpoints.
%cd /content/
!git clone https://github.com/cyrildiagne/3d-photo-inpainting.git
%cd 3d-photo-inpainting
!sh download.sh

# Start Worker

In [0]:
%cd /content/3d-photo-inpainting

import os
import time
import json
import requests
from google.cloud import pubsub
from google.cloud import storage
from IPython.display import clear_output

subscriber = pubsub.SubscriberClient()
subscription_path = subscriber.subscription_path(PROJECT_ID, 'insta3d-sub')

gcp = storage.Client()
bucket = gcp.bucket(BUCKET_NAME)

output_folder = '/content/3d-photo-inpainting'
image_file = output_folder + '/image/img.jpg'
depth_file = output_folder + '/depth/img.npy'
mesh_file = output_folder + '/mesh/img.ply'
video_file = output_folder + '/video/img_circle.mp4'

save_extra = False

if os.path.exists(output_folder + '/image/moon.jpg'):
  os.remove(output_folder + '/image/moon.jpg')

def process(img_url, output_path):
  # Downlaod the image.
  r = requests.get(img_url)
  with open(image_file, 'wb') as f:
    f.write(r.content)
  # Process the image.
  !python main.py --config argument.yml
  # Upload results to storage.
  bucket.blob(output_path).upload_from_filename(video_file)
  if save_extra is True:
    bucket.blob(output_path[:-4] + '.jpg').upload_from_filename(image_file)
    bucket.blob(output_path[:-4] + '.npy').upload_from_filename(depth_file)
    bucket.blob(output_path[:-4] + '.ply').upload_from_filename(mesh_file)
  # Cleanup tmp files
  os.remove(depth_file)
  os.remove(image_file)
  os.remove(video_file)
  os.remove(mesh_file)

while True:
  response = subscriber.pull(subscription_path, max_messages=1)
  if len(response.received_messages):
    for msg in response.received_messages:
      print('Received message.')
      # Load info.
      msg_data = json.loads(msg.message.data.decode('utf-8'))
      # Check if it's not already being processed.
      data_file = 'insta3d/renders/' + msg_data['id'] + '/info.json'
      data = json.loads(bucket.blob(data_file).download_as_string())
      print(msg_data['url'])
      # Acknowledge message.
      subscriber.acknowledge(subscription_path, [msg.ack_id])
      try:
        # Mark as processing in the status file.
        status = { 'id': data['id'], 'status': 'processing' }
        b = bucket.blob(data_file)
        b.cache_control = 'no-cache, max-age=0'
        b.upload_from_string(json.dumps(status))
        # Process.
        img_url = msg_data['url']
        output_file = msg_data['output_file']
        process(img_url, output_file)
        # Mark as ready in the status file.
        status = {
            'id': data['id'],
            'status': 'ready',
            'url': f'https://storage.googleapis.com/{BUCKET_NAME}/{output_file}'
        }
        b = bucket.blob(data_file)
        b.cache_control = 'no-cache, max-age=0'
        b.upload_from_string(json.dumps(status))
        print('Done')
      except Exception as e:
        print(e)
        # Mark as error.
        status = { 'id': data['id'], 'status': 'error', 'error': str(e) }
        b = bucket.blob(data_file)
        b.cache_control = 'no-cache, max-age=0'
        b.upload_from_string(json.dumps(status))
  else:
    clear_output()
    print('Queue is empty. Waiting for new message...')
    time.sleep(1) # wait 1s before checking again

Queue is empty. Waiting for new message...
